# RPS Model Creation

## Preparing the environment

We require a corpus of training data to help build our model.
Quite fortunately for us someone has already created one for us on Kaggle.

[https://www.kaggle.com/alishmanandhar/rock-scissor-paper](https://www.kaggle.com/alishmanandhar/rock-scissor-paper)

A login is required to download this, so we uploaded it to S3 earlier to simplify things.

First thing we need to do is download it to our instance and unzip it.

In [ ]:
import boto3

bucket = 'ac3-sumerian-rps-sagemaker'

boto3.resource('s3').Bucket(bucket).download_file('final.zip', 'final.zip')

!unzip -q -o final.zip

This is python by default.
An ! causes a shell command to run

In [ ]:
!ls

or you could use a bash section instead

In [ ]:
%%bash
ls -l

or even ruby and others!

In [ ]:
%%ruby
5.times { puts 'Hello!' }

By default SageMaker notebook seem to default to using MXNet for keras. We fix this by moving the config file out of the way.

*NB* OK maybe this isn't needed anymore??

In [ ]:
!mv ~/.keras/keras.json keras_tensorflow.json

## Explore the data

Let's take a look at the training and validation data

In [ ]:
%%bash

echo
echo "What is in final"
ls final
echo "========================="
echo
cd final


echo "What is in the train dir"
ls train | head
echo "========================="
echo

echo "What is in the train/c0 dir"
ls train/c0 | head -5
echo
echo "========================="
echo

echo "How many train files"
ls -1R train | wc -l
echo "========================="
echo

echo "How many valid files"
ls -1R valid | wc -l

## Set up python

Import a bunch of libraries we'll need later

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
from keras.applications import MobileNetV2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, TensorBoard

import os

## Do some tuning

### batch_size
  * 32 seems to work - ML Black magic
  * Other good values 64 and 128
  * The bigger the batch size the more memory you need
  
### epochs
  * from testing 20 seems to be more than sufficient to get good results
  * deminishing returns after that
  * The more epochs the longer the training takes
  
### classes
  * We have three
  * Rock, Paper and Scissors

In [ ]:
batch_size = 32
epochs = 20
classes = 3

## Load the model

We want to use MobileNet v2 as our model.
Our input shape is 224, 224, 3
* width
* height
* number of channels
* We don't include the top layer as we are going to replace it

We load the **ImageNet** weights as we are going to do tranfer learning

This will load the model and download the weights.

In [ ]:
conv_base = MobileNetV2(
    input_shape=(224, 224, 3),
    weights='imagenet',
    include_top=False,
)
conv_base.trainable = True
conv_base.summary()

## Prepare a new model

**NB:** This is where the black magic starts

* Create a new sequential model
* Add mobilenet that we loaded in to it
* Flatten
  * flatten stuff???
* Dense
  * Why 256?
  * tanh is one option others are rreli, prelu
* Dense
  * We want only 3 outputs at the end
  * sigmoid is a good function softmax is another
* Print out the summary

In [ ]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='tanh')) # relu (maybe rrelu or prelu will solve dying issue) could also use sigmoid or tanh 
model.add(layers.Dense(classes, activation='sigmoid')) # try softmax
model.summary()

* trainable - renables training for the original model
* We compile the model
  * RMSprop is an opitimizer function which helps 
  * loss - function to measure the loss
  * metrics - the ones we care about

In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5), # Adam?
              loss='categorical_crossentropy',
              metrics=['acc'])


## Prepare the training data
* Load in the training data
* We do lots of permutations on the data to get even more data
* We also load the validation data in.
* We do not manipulate it, only load it to a consistent size

In [ ]:
base_dir = 'final'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)
validation_datagen = ImageDataGenerator(
    rescale=1. / 255,
)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'
)

## Set up checkpoint

Next we set up a checkpoint.
This will save a copy of the model but only if the accuracy is improving.

In [ ]:
filepath="rps-keras-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='val_acc',
    verbose=1,
    save_best_only=True,
    mode='max'
)

# TODO: Add earlystop
callbacks = [checkpoint]

step_size_train = train_generator.n // train_generator.batch_size
step_size_valid = validation_generator.n // validation_generator.batch_size

## Hammer time

We now run the model passing it everything we have prepared earlier

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=step_size_train,
      epochs=epochs,
      callbacks=callbacks,
      validation_data=validation_generator,
      validation_steps=step_size_valid,
      verbose=2)
print('DONE')

## How did we go??

One of the great things about Notebook us being able to display images and matlab plots inline.

Now we can plot how our training went

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt


acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


## Play with our model

Let's run the model against the validation data and check any bad results

In [ ]:
import numpy as np
import matplotlib.image as mpimg

def load_img(filename):
    return mpimg.imread(filename)

# Create a generator for prediction
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
 
# Get the filenames from the generator
fnames = validation_generator.filenames
 
# Get the ground truth from generator
ground_truth = validation_generator.classes
 
# Get the label to class mapping from the generator
label2index = validation_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)
 
errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))
 
# Show the errors
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]
     
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        predictions[errors[i]][pred_class])
     
    original = load_img('{}/{}'.format(validation_dir,fnames[errors[i]]))
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()

## What about the training data

Let's do the same

In [ ]:
import numpy as np
import matplotlib.image as mpimg

def load_image(filename):
    return mpimg.imread('filename')

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
  
# Get the ground truth from generator
ground_truth = validation_generator.classes
 
# Get the label to class mapping from the generator
label2index = validation_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model.predict_generator(train_generator, steps=train_generator.samples/train_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)
 
errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),train_generator.samples))
 


In [ ]:
# Show the errors
for i in range(len(errors)):
  pred_class = np.argmax(predictions[errors[i]])
  pred_label = idx2label[pred_class]
    
  title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
    fnames[errors[i]].split('/')[0],
    pred_label,
    predictions[errors[i]][pred_class])
  
  original = load_img('{}/{}'.format(validation_dir,fnames[errors[i]]))
  plt.figure(figsize=[7,7])
  plt.axis('off')
  plt.title(title)
  plt.imshow(original)
  plt.show()

## We are done

Save the model and upload everything into S3

In [ ]:
model.save('rps-keras-final.hdf5')

!ls -ltr

## Load libraries again in case we are running things out of order
import boto3
from pathlib import Path

bucket = 'ac3-sumerian-rps-sagemaker'

pathlist = Path('.').glob('*.hdf5')
for path in pathlist:
     boto3.resource('s3').Bucket(bucket).upload_file(str(path), 'keras/{}'.format(str(path)))

# GO BACK TO YOUR ORIGINAL SCRIPT AND STOP THE NOTEBOOK